## AWS with Spark

In [47]:
import configparser
import os
import sys
os.environ['PYSPARK_SUBMIT_ARGS'] = "--master mymaster --total-executor 2 --conf 'spark.driver.extraJavaOptions=-Dhttp.proxyHost=proxy.mycorp.com-Dhttp.proxyPort=1234 -Dhttp.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 -Dhttps.proxyHost=proxy.mycorp.com -Dhttps.proxyPort=1234 -Dhttps.nonProxyHosts=localhost|.mycorp.com|127.0.0.1 pyspark-shell'"
import findspark
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['KEY']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['SECRET']

In [42]:
os.environ['SPARK_HOME'] = "/usr/local/Cellar/apache-spark/2.4.3"
os.environ['JAVA_HOME'] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_65.jdk/Contents/Home"
os.environ['PYSPARK_SUBMIT_ARGS']= "--master local[2] pyspark-shell"

### Create spark session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .appName("demo")\
                     .getOrCreate()

## Part 1: Schema on Read

### Read the file

In [ ]:
df = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv",sep=";", inferSchema=True, header=True)

df.printSchema()
df.show(5)

### Convert string to timestamp

In [ ]:
import pyspark.sql.functions as F
dfPayment = df.withColumn("payment_date", F.to_timestamp("payment_date"))
dfPayment.printSchema()
dfPayment.show(5)

### Extract the month

In [ ]:
dfPayment = dfPayment.withColumn("month", F.month("payment_date"))
dfPayment.show(5)

### Get monthly revenue

In [ ]:
dfPayment.createOrReplaceTempView("payment")
spark.sql("""
    SELECT month, sum(amount) as revenue
    FROM payment
    GROUP by month
    order by revenue desc
""").show()

## Pre-determined schema

### Fix the schema

In [ ]:
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
paymentSchema = R([
    Fld("payment_id",Int()),
    Fld("customer_id",Int()),
    Fld("staff_id",Int()),
    Fld("rental_id",Int()),
    Fld("amount",Dbl()),
    Fld("payment_date",Date()),
])

### Read the file

In [ ]:
dfPaymentWithSchema = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv",sep=";", schema=paymentSchema, header=True)

In [ ]:
dfPaymentWithSchema.printSchema()
df.show(5)

### Get monthly revenue

In [ ]:
dfPaymentWithSchema.createOrReplaceTempView("payment")
spark.sql("""
    SELECT month(payment_date) as m, sum(amount) as revenue
    FROM payment
    GROUP by m
    order by revenue desc
""").show()